In [ ]:
import json

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F

import numpy as np
import pandas as pd

# Data import

In [ ]:
with open("example-data.json", "r") as filehandle:
    data = json.load(filehandle)

# Data Parsing

In [ ]:
data_parsed = [entry.replace(", ", ",") for entry in data]
data_parsed = [entry.split(",") for entry in data_parsed]
data_parsed = pd.DataFrame.from_dict(data_parsed)
data_parsed.columns = ["timestamp", "x", "temp", "humid"]
# Convert temp and humid to numeric
data_parsed["temp"] = data_parsed["temp"].astype(float)
data_parsed["humid"] = data_parsed["humid"].astype(float)
# Last row is empty
data_parsed = data_parsed[:-1]
# Remove X
data_parsed.drop(columns="x", inplace=True)
# Convert to datetime
data_parsed["timestamp"] = pd.to_datetime(data_parsed["timestamp"])


# Data Stats

In [ ]:
# Time range
print(data_parsed["timestamp"].min(), data_parsed["timestamp"].max())
# Mean Temp per weekday
print(data_parsed.groupby(
    [data_parsed["timestamp"].dt.weekday])["temp"].mean())
# Mean Humid per weekday
print(data_parsed.groupby(
    [data_parsed["timestamp"].dt.weekday])["humid"].mean())

# Feature Enhancement

In [ ]:
# Add hour
data_parsed["hour"] = data_parsed["timestamp"].dt.hour
# Add day of year
data_parsed["day_of_year"] = data_parsed["timestamp"].dt.day_of_year
# Add weekday
data_parsed["weekday"] = data_parsed["timestamp"].dt.weekday

# ML

In [ ]:
class FFNModel(nn.Module):

    def __init__(self):
        super(FFNModel, self).__init__()
        self.fc1 = nn.Linear(3, 9)
        self.fc2 = nn.Linear(9, 1)            

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

startnet = FFNModel()
optimizer = optim.Adam(startnet.parameters(), lr=0.001)

In [ ]:
def training_loop(n_epochs, optimiser, model, loss_fn, train_dl, val_dl):
    for epoch in range(1, n_epochs + 1):
        model.train()
        for i, data in enumerate(train_dl):
            optimiser.zero_grad() # set gradients to zero
            inputs, targets = data
            output_train = model(inputs) # forwards pass
            loss_train = loss_fn(output_train, targets) # calculate loss
            loss_train.backward() # backwards pass
            optimiser.step() # update model parameters

        model.eval()
        for i, data in enumerate(val_dl):
            inputs, targets = data
            output_val = model(inputs)
            loss_val = loss_fn(output_val, targets)
        if epoch == 1 or epoch % 100 == 0:
            print(f"Epoch {epoch}, Training loss {loss_train.item():.4f},"
                f" Validation loss {loss_val.item():.4f}")

In [ ]:
# Train Val Split
train = data_parsed[0:int(len(data_parsed)/1.5)]
val = data_parsed[int(len(data_parsed)/1.5):]
# Full Training
train = data_parsed

class MyDataset(torch.utils.data.Dataset):

  def __init__(self, df):
 
    x = df[["hour", "weekday", "day_of_year"]].values
    y = df[["temp"]].values

    self.x_train=torch.tensor(x, dtype=torch.float32)
    self.y_train=torch.tensor(y, dtype=torch.float32)

  def __len__(self):
    return len(self.y_train)
  
  def __getitem__(self, idx):
    return self.x_train[idx], self.y_train[idx]

In [ ]:
train_dl  = torch.utils.data.DataLoader(MyDataset(train), batch_size=10, shuffle=True)
val_dl    = torch.utils.data.DataLoader(MyDataset(val), batch_size=10, shuffle=True)

In [ ]:
training_loop(
    n_epochs = 100, 
    optimiser = optimizer,
    model = startnet,
    loss_fn = nn.MSELoss(),
    train_dl = train_dl,
    val_dl = val_dl,
    )

In [ ]:
for name, param in startnet.named_parameters():
    print(name, param)

In [ ]:
startnet.eval()
for i in range(12, 23):
    for j in range(3, 6):
        for k in range(310, 320):
            print(startnet(torch.tensor([float(i), j, k])).tolist()[0])

In [ ]:
torch.save(startnet.state_dict(), "temperature_fnn.model") 